In [2]:
import os
import openai
from dotenv import load_dotenv
import requests
from bs4 import BeautifulSoup
import warnings
import re

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

def url_to_string(url):
    warnings.filterwarnings('ignore', message='Unverified HTTPS request')
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.text, "html.parser")
    return soup.get_text()

def split_text(text, chunk_size=1000):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        if end < len(text) and text[end] not in '.?!':
            while end < len(text) and text[end] not in '.?!':
                end += 1
        chunks.append(text[start:end + 1].strip())
        start = end + 1
    return chunks

def openai_edit(text, i = "Rewrite it in narrative style") :
    response = openai.Edit.create(
      model = "text-davinci-edit-001",
      input = text,
      instruction = i
    )   
    return response['choices'][0]['text']

def openai_complete(text, i = "in a narrative style rewrite text below") :
    response = openai.Completion.create(
        model = "text-davinci-003",
        prompt = i+"\n"+ text +"",
        temperature = 0.5,
        max_tokens = 2048,
        top_p = 1,
        frequency_penalty = 0.2,
    )       
    return response['choices'][-1]['text']

def rewrite_text(chunks, func):
    rewritten_text = ""
    for chunk in chunks:
        chunk_rewritten_text = func(chunk)
        rewritten_text += chunk_rewritten_text
    return rewritten_text
    
text = url_to_string('http://socalnews.com/la-story-3.html')
text_chunks = split_text(text)

small_text_example = "The Burbank-based Walt Disney Co. will slash 7,000 jobs as part of a $5.5 billion cost-cutting effort, CEO Bob Iger announced Wednesday.The possibility of layoffs at Disney has been rumored for weeks, with Iger regaining his footing in the CEO’s office following the surprise ouster of CEO Bob Chapek in November. Iger broke the news during an earnings call Wednesday afternoon, following the release of the company’s first-quarter earnings report. Iger said he is targeting $5.5 billion in cost savings “across the company.”"
print(text)


Bass Inauguration, 3rd LdDate: 12-11-2022 8:50 PM - Word Count: 1283Bass Inauguration, 3rd Ld
   Karen Bass Inaugurated as Mayor, to Hold News Conference
   Eds: UPDATES with Bass to hold news conference, in 7th graf. Bass'
office can be reached at press@karenbass.com.
   By ERIC HE
   City News Service
   LOS ANGELES (CNS) - Karen Bass was sworn in as the 43rd mayor of Los
Angeles today in a historic inauguration at the Microsoft Theater, becoming the
first woman and second Black person to lead the city.
   Bass was sworn in by Vice President Kamala Harris, a former California
senator and the first woman to serve as the nation's second-in-command.
   Nearly every major city official, along with Gov. Gavin Newsom,
attended the ceremony.
   ``Making history with each of you today is a monumental moment in my
life and for Los Angeles,'' Bass said in her speech.
   Bass addressed what she described as an ``inflection point in our
history,'' with issues including ``the pandemic, the rapidl

In [47]:
# new_text = rewrite_text(text_chunks, openai_edit)
edit_model = openai_edit(small_text_example)
print("Example #1\nModel: Edit\n\n" +edit_model)

Example #1
Model: Edit

The Burbank-based Walt Disney Co. announced Wednesday it will slash 7,000 jobs as part of a $5.5 billion cost-cutting effort.
The possibility of layoffs had been rumored for weeks, with CEO Bob Iger regaining his footing in the CEO’s office following the surprise ouster of former CEO Bob Chapek in November.
Iger announced this during an earnings call Wednesday, following the release of the company’s first-quarter earnings report.
Iger said he is targeting $5.5 billion in cost savings “across the company.”



In [48]:
edit_model = openai_edit(small_text_example)
print("Example #2\nModel: Edit\n\n" +edit_model)

Example #2
Model: Edit

Earlier today, the Burbank-based Walt Disney Co. announced a plan to slash 7,000 jobs as part of a $5.5 billion cost-cutting effort.
CEO Bob Iger announced the plan during the company’s first-quarter earnings call on Wednesday.
The possibility of layoffs at Disney has been rumored for weeks, with Iger regaining his footing in the CEO’s office following the surprise ouster of CEO Bob Chapek in November.
Iger said he is targeting $5.5 billion in cost savings “across the company.”



In [56]:
complete_model = openai_complete(small_text_example)
print("Example #1\nModel: Complete" +complete_model)

Example #1
Model: Complete

On Wednesday, Bob Iger, CEO of the Burbank-based Walt Disney Company, confirmed weeks of speculation that the company would be cutting 7,000 jobs as part of a $5.5 billion cost-cutting effort. Iger made the announcement during an earnings call following the release of the company’s first-quarter earnings report. He stated that these reductions will be spread “across the company” and are necessary in order to reach their goal of $5.5 billion in cost savings. This news comes shortly after Iger took back his role as CEO following the unexpected departure of Bob Chapek in November.


In [55]:
complete_model = openai_complete(small_text_example)
print("Example #2\nModel: Complete" +complete_model)

Example #2
Model: Complete

Bob Iger, CEO of the Burbank-based Walt Disney Co., announced on Wednesday that the company will lay off 7,000 employees in a $5.5 billion cost-cutting effort. This news follows weeks of speculation and rumors after Bob Chapek was unexpectedly removed from the CEO's office in November. Iger made the announcement during an earnings call after the release of the company's first-quarter earnings report. He stated that the company is aiming for $5.5 billion in savings "across the company."


In [29]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk
# import spacy
import en_core_web_sm
from pprint import pprint

def text_to_ents(text):
    nlp = en_core_web_sm.load()
    doc = nlp(text)
    return doc.ents

entities = text_to_ents(text)
pprint([(X.text, X.label_) for X in entities])

[('Bass Inauguration', 'PERSON'),
 ('12', 'CARDINAL'),
 ('8:50 PM', 'TIME'),
 ('3rd', 'ORDINAL'),
 ('Hold News Conference\r\n\xa0\xa0 Eds', 'ORG'),
 ('Bass', 'PERSON'),
 ('7th', 'ORDINAL'),
 ('Bass', 'PERSON'),
 ('ERIC HE\r\n\xa0\xa0 City News Service', 'PERSON'),
 ('LOS ANGELES', 'GPE'),
 ('CNS', 'ORG'),
 ('Karen Bass', 'PERSON'),
 ('43rd', 'ORDINAL'),
 ('Los\r\nAngeles', 'GPE'),
 ('today', 'DATE'),
 ('the Microsoft Theater', 'ORG'),
 ('first', 'ORDINAL'),
 ('second', 'ORDINAL'),
 ('Bass', 'PERSON'),
 ('Kamala Harris', 'PERSON'),
 ('California', 'GPE'),
 ('first', 'ORDINAL'),
 ('second', 'ORDINAL'),
 ('Gavin Newsom', 'PERSON'),
 ('today', 'DATE'),
 ('Los Angeles', 'GPE'),
 ('Bass', 'PERSON'),
 ('Bass', 'PERSON'),
 ('40,000', 'CARDINAL'),
 ('Angelenos', 'PERSON'),
 ('Bass', 'PERSON'),
 ('9 a.m.', 'TIME'),
 ('Monday', 'DATE'),
 ('first', 'ORDINAL'),
 ('Bass', 'PERSON'),
 ('Bass', 'PERSON'),
 ('17,000', 'CARDINAL'),
 ('her first\r\nyear', 'DATE'),
 ('Los Angeles', 'GPE'),
 ('Bass', 'PERS

In [4]:
from collections import Counter
labels = [x.label_ for x in entities]
Counter(labels)

Counter({'PERSON': 52,
         'CARDINAL': 9,
         'TIME': 3,
         'ORDINAL': 11,
         'ORG': 17,
         'GPE': 15,
         'DATE': 33,
         'FAC': 3,
         'WORK_OF_ART': 2,
         'NORP': 1,
         'EVENT': 1})

In [5]:
items = [x.text for x in entities]
Counter(items).most_common(3)

[('Bass', 29), ('today', 7), ('first', 6)]

In [104]:
import wandb

run = wandb.init(project='HeywireAI', entity="heywire")
wandb.config = {
  "learning_rate": 0.001,
  "epochs": 100,
  "batch_size": 128
}
prediction_table = wandb.Table(columns=["original_story", "prompt", "completion"])

def generate_story(article_text, entities, summary):

    mod = "text-davinci-003"
    prompt = "Generate a story that involves the following entities and origin example:"
    
    completions = openai.Completion.create(
        model=mod,
        prompt=prompt,
        max_tokens=1024,
        stop=None,
        temperature=0.5,
    )

    entity_string = "\n".join([f"- {entity.text}: {entity.label_}" for entity in entities])
    
    if summary:
        story_prompt = f"Original example: {summary}\n Entities: {entity_string}\n\nBegin story:"
    else:
        story_prompt = f"{entity_string}\n\nBegin story:"
    
    story_completions = openai.Completion.create(
        model=mod,
        prompt=story_prompt,
        max_tokens=int(4096 - (len(story_prompt) / 3)),
        n=1,
        stop=None,
        temperature=0.5,
    )
    
    full_prompt = f"{prompt}\n{story_prompt}:"
    
    prediction_table.add_data(article_text, full_prompt, story_completions['choices'][0]['text'])
    
    return story_completions['choices'][0]['text']

In [40]:
articles = [
    'https://socalnews.com/la-story-3.html',
    'https://socalnews.com/la-story-4.html',
    'https://socalnews.com/la-story-5.html',
    'https://socalnews.com/la-story-6.html',
    'https://socalnews.com/orange-story3.htm',
    'https://socalnews.com/orange-story4.htm'
]

def stories_run(stories):
    for story in stories:
        article_text = url_to_string(story)
        ents = text_to_ents(article_text)
        new_story = generate_story(article_text, ents)
        print(new_story)
    
    wandb.log({'predictions': prediction_table})
    wandb.finish()
    return

stories_run(articles)



Karen Bass was inaugurated as the 43rd mayor of Los Angeles today, at the Microsoft Theater. Bass, first elected to the Los Angeles City Council in 2007, and re-elected to the council in 2011 and 2015, was sworn in by California Senator Kamala Harris during a ceremony attended by Governor Gavin Newsom.

Bass addressed a crowd of 40,000 Angelenos gathered at City Hall, saying she was humbled to have been chosen as the city's first female African American mayor. She noted that the inauguration marked the 241-year anniversary of the first day of the City of Los Angeles.

The ceremony featured musical performances by Mark Gonzalez, Stevie Wonder, Chloe Bailey, and Las Cafeteras, as well as a poem by Amanda Gorman, the first person to hold the title of Youth Poet Laureate of the United States. Gorman read her poem at the inauguration of President Joe Biden's on Sunday.

Bass, who was elected mayor of Los Angeles on November 8, 2020, also noted that the inauguration was taking place after 



Robert Luna was appointed as the new sheriff of Los Angeles County on Saturday, replacing Jim McDonnell, who held the post for the past four years. Luna, who was previously the Long Beach police chief, was sworn in during a ceremony held at the Hall of Administration in downtown Los Angeles.

The ceremony was attended by more than 18,000 people, including Los Angeles Mayor Alex Villanueva and Long Beach Mayor Robert Garcia. Luna was also joined by Supervisor Holly Mitchell and other county officials.

In his speech, Luna thanked McDonnell for his service to the county and said he was honored to be the first sheriff in roughly a century to come from the Long Beach Police Department. He also thanked the people of Los Angeles County for their support in the Nov. 8 election, when he was elected to the post with more than 60 percent of the vote.

Luna then discussed his plans for the department, which include increasing the number of deputies to more than 18,000 and implementing new tan-a

In [94]:
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

def get_summary(article, count):
    LANGUAGE = "english"
    SENTENCES_COUNT = count

    if __name__ == "__main__":
        parser = PlaintextParser.from_string(article, Tokenizer(LANGUAGE))
        stemmer = Stemmer(LANGUAGE)

        summarizer = Summarizer(stemmer)
        summarizer.stop_words = get_stop_words(LANGUAGE)

        summary = " ".join(str(sentence) for sentence in summarizer(parser.document, SENTENCES_COUNT))
        return summary

s = get_summary(text, 8)
s

"The emergency declaration will ``recognize the severity of our crisis and break new ground to maximize our ability to urgently move people inside, and do so for good. Sunday's ceremony was initially scheduled to take place outside City Hall, but rain in the forecast led to a venue shift indoors. Surprise performers at the inauguration included Stevie Wonder, who sang ``Keep Our Love Alive'' and ``Living for the City,'' and Chloe Bailey and Las Cafeteras. Amanda Gorman, the first National Youth Poet Laureate, who also delivered a poem at President Joe Biden's inauguration, said in her reading Sunday that ``the time of never before is officially past.'' Bass' father fled the Jim Crow south during the great African American migration after World War II and found work as a letter carrier. ``When I think about the dreams of working people today, I reflect on the fact that my mother and father were able to buy a home in Los Angeles for their family of six with one paycheck,'' Bass said. Bas

In [102]:
new_story_summary = generate_story(text, entities, s)
new_story_summary

1350.0


'\nKaren Bass was inaugurated as the 43rd mayor of Los Angeles on Sunday at the Microsoft Theater, making her the first woman and second African American to hold the office. Bass was sworn in by California\'s first and second female U.S. senators, Kamala Harris and Dianne Feinstein, and was joined by Gov. Gavin Newsom and other state and local officials. \n\nBass, who was elected in November with more than 40,000 votes, thanked Angelenos for their support. “It’s been nine days since the election and I’m already getting to work," she said, adding that she was already preparing for her first day in office on Monday at 9 a.m.\n\nBass, who previously served in the state legislature and the U.S. House, is the first mayor of Los Angeles in nearly a decade who is not Eric Garcetti. After being elected in 2013, Garcetti was reelected in 2017 and was term-limited from running again. \n\nBass won the election with a campaign focused on economic and racial justice, and on Sunday she promised to c

In [103]:
wandb.log({'predictions': prediction_table})
wandb.finish()
print(new_story_summary)


Karen Bass was inaugurated as the 43rd mayor of Los Angeles on Sunday at the Microsoft Theater, making her the first woman and second African American to hold the office. Bass was sworn in by California's first and second female U.S. senators, Kamala Harris and Dianne Feinstein, and was joined by Gov. Gavin Newsom and other state and local officials. 

Bass, who was elected in November with more than 40,000 votes, thanked Angelenos for their support. “It’s been nine days since the election and I’m already getting to work," she said, adding that she was already preparing for her first day in office on Monday at 9 a.m.

Bass, who previously served in the state legislature and the U.S. House, is the first mayor of Los Angeles in nearly a decade who is not Eric Garcetti. After being elected in 2013, Garcetti was reelected in 2017 and was term-limited from running again. 

Bass won the election with a campaign focused on economic and racial justice, and on Sunday she promised to continue t